In [1]:
import os
from llm_workflow.hugging_face import HuggingFaceEndpointChat, get_tokenizer, num_tokens
from IPython.display import Markdown, display
from dotenv import load_dotenv
load_dotenv()

tokenizer = get_tokenizer('meta-llama/Llama-2-7b-chat-hf')
def cal_num_tokens(text: str) -> int:
    return num_tokens(text, tokenizer)

model = HuggingFaceEndpointChat(
    endpoint_url=os.getenv('HUGGING_FACE_ENDPOINT_LLAMA2_7B'),
    token_calculator=cal_num_tokens,
    streaming_callback=lambda x: print(x.response, end='||', flush=True)
)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model("Hi, my name is Shane. What's your name?")

Hello Shane! My name is Assistant,|| I'm here to help you with any questions|| or tasks you may have. How can I assist|| you today?||

"Hello Shane! My name is Assistant, I'm here to help you with any questions or tasks you may have. How can I assist you today?"

In [3]:
model.total_tokens

91

In [4]:
model("What's my name?")

Ah, a great question! Your name is Sh||ane. 😊||

'Ah, a great question! Your name is Shane. 😊'

---

In [5]:
model = HuggingFaceEndpointChat(
    endpoint_url=os.getenv('HUGGING_FACE_ENDPOINT_LLAMA2_7B'),
    streaming_callback=lambda x: print(x.response, end='||', flush=True)
)

In [6]:
response = model("Write a python function to sum two numbers.")
display(Markdown(response))

Of course! Here is a simple Python function to|| sum two numbers:
```
def sum(||a, b):
    return a + b||
```
Explanation:

*|| The function name is `sum`, which is a|| common name for a function that calculates the sum|| of two numbers.
* The function takes two|| arguments, `a` and `b`, which|| are the two numbers to be summed.
||* The function returns the sum of `a`|| and `b` using the `+` operator||.
* The function is defined inside a `||def` block, which is used to define a|| new function.
* The `return` statement|| at the end of the function defines the value that|| the function will return when it is called. In|| this case, the function will return the sum of|| `a` and `b`.
Example usage||:
```
a = 3
b|| = 5
result = sum(a,|| b)
print(result) # Output:|| 8
```
I hope this helps!|| Let me know if you have any questions.||

Of course! Here is a simple Python function to sum two numbers:
```
def sum(a, b):
    return a + b
```
Explanation:

* The function name is `sum`, which is a common name for a function that calculates the sum of two numbers.
* The function takes two arguments, `a` and `b`, which are the two numbers to be summed.
* The function returns the sum of `a` and `b` using the `+` operator.
* The function is defined inside a `def` block, which is used to define a new function.
* The `return` statement at the end of the function defines the value that the function will return when it is called. In this case, the function will return the sum of `a` and `b`.
Example usage:
```
a = 3
b = 5
result = sum(a, b)
print(result) # Output: 8
```
I hope this helps! Let me know if you have any questions.

In [7]:
response = model("Update the function to use a doc-string and type hints.")
display(Markdown(response))

Of course! Here is an updated version of the|| function that includes a doc-string and type hints||:
```
def sum(a: int||, b: int) -> int:
   || """
    Calculates the sum of two numbers||.
    :param a: The first number|| to be summed.
    :param b||: The second number to be summed.
||    :return: The sum of a and b||.
    """
    return a + b||
```
Explanation:

*|| The doc-string is a brief description of the|| function and its purpose. It is placed above the|| function definition and is enclosed in triple quotes (`||"""`).
* The type hints are|| used to specify the data types of the function arguments|| and return value. They are placed above the function|| definition and are enclosed in parentheses `()` .||
* The function signature is the combination of the|| function name, arguments, and return type. In|| this case, the function takes two `int`|| arguments (`a` and `b`) and returns|| an `int` value (`result`).
*|| The function body is the code that is executed when|| the function

Of course! Here is an updated version of the function that includes a doc-string and type hints:
```
def sum(a: int, b: int) -> int:
    """
    Calculates the sum of two numbers.
    :param a: The first number to be summed.
    :param b: The second number to be summed.
    :return: The sum of a and b.
    """
    return a + b
```
Explanation:

* The doc-string is a brief description of the function and its purpose. It is placed above the function definition and is enclosed in triple quotes (`"""`).
* The type hints are used to specify the data types of the function arguments and return value. They are placed above the function definition and are enclosed in parentheses `()` .
* The function signature is the combination of the function name, arguments, and return type. In this case, the function takes two `int` arguments (`a` and `b`) and returns an `int` value (`result`).
* The function body is the code that is executed when the function is called. In this case, the function simply calculates the sum of `a` and `b` using the `+` operator and returns the result.
Example usage:
```
a = 3
b = 5
result = sum(a, b)
print(result) # Output: 8
```
I hope this helps! Let me

---